In [35]:
import json
import pandas as pd
import glob
import datetime
from datetime import date, timedelta
import openpyxl
import pathlib
import numpy as np
from datetime import datetime
import os
import re
from pathlib import Path
import os.path
import time
from collections import OrderedDict
import csv
import shutil
import polars as pl
import fastexcel

In [36]:
def convert_datetime(a):
    my_format="%Y-%m-%d %H:%M:%S"
    date_formats = ["%Y-%m-%d %H:%M:%S.%f", 
                    "%Y-%m-%d %H:%M:%S", 
                    "%m/%d/%Y %H:%M",
                    "%m-%d-%Y %H:%M", 
                    "%m/%d/%Y", 
                    "%Y-%m-%d", 
                    "%H:%M.%f",
                    "%m/%d/%Y %H:%M:%S"] 
    for fmt in date_formats:
        try:
            return datetime.strptime(str(a), fmt).strftime("%Y-%m-%d %H:%M:%S")
        except ValueError:
            pass
    try:
        return (datetime(1899, 12, 30) + timedelta(days=float(a))).strftime("%Y-%m-%d %H:%M:%S")
    except ValueError:
        pass
    if not re.match("\d+:\d+:\d+(\.\d+)?", str(a)):
        return str(a)
    return  
def convert_date(a):
    date_formats = ["%Y-%m-%d %H:%M:%S.%f", 
                    "%Y-%m-%d %H:%M:%S", 
                    "%m/%d/%Y %H:%M",
                    "%m-%d-%Y %H:%M", 
                    "%m/%d/%Y", 
                    "%Y-%m-%d", 
                    "%H:%M.%f",
                    "%m/%d/%Y %H:%M:%S"] 
    for fmt in date_formats:
        try:
            return datetime.strptime(str(a), fmt).strftime("%Y-%m-%d")
        except ValueError:
            pass
    try:
        return (datetime(1899, 12, 30) + timedelta(days=float(a))).strftime("%Y-%m-%d")
    except ValueError:
        pass
    if not re.match("\d+:\d+:\d+(\.\d+)?", str(a)):
        return str(a)
    return  
def convert_to_datetime(struct_time):
    return datetime(*struct_time[:6])
def input_data(data_dir):
    list_files = []
    for filename in pathlib.Path(data_dir).glob('**/*.*'):
        file_suffixes = filename.suffixes
        if file_suffixes[-1].lower() in ['.xlsx', '.csv']:
            export_time = os.path.getmtime(filename)
            export_time_datetime = convert_to_datetime(time.localtime(export_time))
            
            if file_suffixes[-1].lower() == '.xlsx':
                file_name = filename.stem
                dfs = pd.read_excel(filename, sheet_name=None, skiprows=0, na_values=None)
                for sheet_name, df in dfs.items():
                    df['sheet_name'] = file_name 
                    df['Export time'] = export_time_datetime
                    list_files.append(df)
                    
            elif file_suffixes[-1].lower() == '.csv':
                file_name = filename.stem
                df = pd.read_csv(filename, skiprows=0, na_values=None)
                df['sheet_name'] = file_name
                df['Export time'] = export_time_datetime
                list_files.append(df)
                
    df_list = pd.concat(list_files, axis=0, ignore_index=True)
    return df_list

<>:20: SyntaxWarning: invalid escape sequence '\d'
<>:41: SyntaxWarning: invalid escape sequence '\d'
<>:20: SyntaxWarning: invalid escape sequence '\d'
<>:41: SyntaxWarning: invalid escape sequence '\d'
C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_11796\3603552263.py:20: SyntaxWarning: invalid escape sequence '\d'
  if not re.match("\d+:\d+:\d+(\.\d+)?", str(a)):
C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_11796\3603552263.py:41: SyntaxWarning: invalid escape sequence '\d'
  if not re.match("\d+:\d+:\d+(\.\d+)?", str(a)):


In [39]:
IEX_Input = input_data(r'1. DAILY RAWDATA/3. IEX/1. VIETNAM')

['Agent Schedules',
 '__UNNAMED__1',
 '__UNNAMED__2',
 '__UNNAMED__3',
 '__UNNAMED__4',
 '__UNNAMED__5',
 '__UNNAMED__6',
 '__UNNAMED__9',
 'sheet_name',
 'Export time']

In [41]:
columns_to_remove = ["Unnamed: 0", "Unnamed: 8", "Unnamed: 9", "Unnamed: 5", "Unnamed: 11", "Unnamed: 12"]

IEX = IEX_Input[[col for col in IEX_Input.columns if col not in columns_to_remove]]
IEX = IEX.rename(columns={
    "Unnamed: 1": "Agent",
    "Unnamed: 2": "Date",
    "Unnamed: 3": "Start_Shift",
    "Unnamed: 4": "End_Shift",
    "Unnamed: 6": "Scheduled Activity",
    "Unnamed: 7": "Start_Action",
    "Unnamed: 10": "End_Action"
})

IEX['Generate Date'] = np.where(
    IEX['Agent'].str.contains("Generation Date: ", na=False),
    IEX['Agent'].str.extract(r'Generation Date: (.+)')[0],
    np.nan
)
IEX['Generate Date'] = IEX['Generate Date'].fillna(method='bfill')
IEX_edit = IEX.copy()
IEX_edit['Agent'] = IEX_edit['Agent'].fillna(method='ffill')
offTable_save = IEX_edit
IEX_edit = IEX_edit[(IEX_edit['Start_Shift'] != "Off") & (IEX_edit['Date'] != "Date")]
IEX_edit = IEX_edit[~(IEX_edit['Date'].isna() & IEX_edit['Scheduled Activity'].isna())]
IEX_edit

ColumnNotFoundError: 'rename' on column: '__UNNAMED__7' is invalid

Schema at this point: Schema:
name: Agent Schedules, field: String
name: __UNNAMED__1, field: String
name: __UNNAMED__2, field: String
name: __UNNAMED__3, field: String
name: __UNNAMED__4, field: String
name: __UNNAMED__5, field: String
name: __UNNAMED__6, field: String
name: __UNNAMED__9, field: String
name: sheet_name, field: String
name: Export time, field: Datetime(Microseconds, None)


Resolved plan until failure:

	---> FAILED HERE RESOLVING THIS_NODE <---
DF ["Agent Schedules", "__UNNAMED__1", "__UNNAMED__2", "__UNNAMED__3"]; PROJECT */10 COLUMNS; SELECTION: None

In [5]:
offTable = offTable_save[offTable_save['Start_Shift'] == "Off"].reset_index(drop=True)

condition = (IEX_edit['Date'] != "Date") & (IEX_edit['Start_Shift'] != "Off")
IEX_edit = IEX_edit[condition].reset_index(drop=True)
IEX_edit[['Date', 'Start_Shift', 'End_Shift']] = IEX_edit[['Date', 'Start_Shift', 'End_Shift']].fillna(method='ffill')

condition2 = (IEX_edit['Agent'].str.contains("Agent: ", na=False)) | (IEX_edit['Agent'].isna())
IEX_edit = IEX_edit[condition2].reset_index(drop=True)

offTable['Scheduled Activity'] = offTable['Scheduled Activity'].fillna(offTable['Start_Shift'])
offTable['Start_Shift'] = offTable['Start_Shift'].replace('Off', np.nan)

IEX_edit = IEX_edit[IEX_edit['Scheduled Activity'].notna()].reset_index(drop=True)
combined_df = pd.concat([offTable, IEX_edit], axis=0, ignore_index=True)

C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25864\1879448329.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  IEX_edit[['Date', 'Start_Shift', 'End_Shift']] = IEX_edit[['Date', 'Start_Shift', 'End_Shift']].fillna(method='ffill')
C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25864\1879448329.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  offTable['Start_Shift'] = offTable['Start_Shift'].replace('Off', np.nan)


In [6]:
combined_df['Export time'] = pd.to_datetime(combined_df['Export time'], errors='coerce')
combined_df['Generate Date'] = pd.to_datetime(combined_df['Generate Date'], errors='coerce')
combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce')
combined_df['Year'] = combined_df['Date'].dt.year
combined_df['Month'] = combined_df['Date'].dt.strftime('%b-%y')
combined_df['IEX_ID'] = combined_df['Agent'].str.extract(r'(\d+)', expand=False).astype(int)

max_generate_date_df = combined_df.groupby(['IEX_ID','Date'])['Generate Date'].max().reset_index()
max_generate_date_df = max_generate_date_df.rename(columns={'Generate Date': 'Max Generate Date'})

max_generate_date_with_details = pd.merge(combined_df, max_generate_date_df, on=['Date','IEX_ID'], how='left')
filtered_max_generate_date = max_generate_date_with_details[max_generate_date_with_details['Generate Date'] == max_generate_date_with_details['Max Generate Date']]

filtered_max_generate_date['Scheduled'] = filtered_max_generate_date.groupby(['Date', 'IEX_ID'])['Scheduled Activity'].transform(lambda x: 1 if any(x.isin(['Open Time', 'Extra Hours', 'No Call/No Show', 'PTO'])) else 0)

C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25864\2330591367.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_df['Generate Date'] = pd.to_datetime(combined_df['Generate Date'], errors='coerce')
C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25864\2330591367.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce')


In [7]:
filtered_max_generate_date['Start_Shift'] = pd.to_datetime(filtered_max_generate_date['Start_Shift'], format='%I:%M %p', errors='coerce').dt.time
filtered_max_generate_date['End_Shift'] = pd.to_datetime(filtered_max_generate_date['End_Shift'], format='%I:%M %p', errors='coerce').dt.time
filtered_max_generate_date['Start_Action'] = pd.to_datetime(filtered_max_generate_date['Start_Action'], format='%I:%M %p', errors='coerce').dt.time
filtered_max_generate_date['End_Action'] = pd.to_datetime(filtered_max_generate_date['End_Action'], format='%I:%M %p', errors='coerce').dt.time

def create_datetime(row, time_column, shift_column=None):
    if row['Scheduled'] == 0:
        return pd.NaT
    if pd.isnull(row['Date']) or pd.isnull(row[time_column]):
        return pd.NaT
    time_value = row[time_column]
    if shift_column and pd.isnull(row[shift_column]):
        return pd.NaT
    if shift_column and time_value < row[shift_column]:
        return pd.Timestamp.combine(row['Date'], time_value) + pd.DateOffset(days=1)
    return pd.Timestamp.combine(row['Date'], time_value)

filtered_max_generate_date = filtered_max_generate_date.drop(filtered_max_generate_date[(filtered_max_generate_date['Scheduled Activity'].isin(['Lunch', 'Break','Training'])) & (filtered_max_generate_date['Scheduled'] == 0)].index)

filtered_max_generate_date['Datetime_Start_Action'] = filtered_max_generate_date.apply(lambda row: create_datetime(row, 'Start_Action', 'Start_Shift'), axis=1)
filtered_max_generate_date['Datetime_End_Action'] = filtered_max_generate_date.apply(lambda row: create_datetime(row, 'End_Action', 'Start_Shift'), axis=1)

filtered_fluctuate_shifts = filtered_max_generate_date[
    filtered_max_generate_date['Scheduled Activity'].isin(["Open Time", "Extra Hours"]) | 
    filtered_max_generate_date['Scheduled Activity'].str.contains("Training", na=False)
]
min_max_fluctuate_shifts = filtered_fluctuate_shifts.groupby(['Date', 'IEX_ID']).agg(
    Datetime_Fluctuate_Start_Shift=('Datetime_Start_Action', 'min'),
    Datetime_Fluctuate_End_Shift=('Datetime_End_Action', 'max')
).reset_index()
min_max_fluctuate_shifts['Fluctuate Shift'] = (min_max_fluctuate_shifts['Datetime_Fluctuate_Start_Shift'].dt.strftime('%H%M') + '-' + min_max_fluctuate_shifts['Datetime_Fluctuate_End_Shift'].dt.strftime('%H%M'))
new_table_with_fluctuate_shifts_col = pd.merge(filtered_max_generate_date, min_max_fluctuate_shifts, on=['Date', 'IEX_ID'], how='left')


min_max_first_shifts = filtered_max_generate_date.groupby(['Date', 'IEX_ID']).agg(
    Datetime_First_Start_Shift=('Datetime_Start_Action', 'min'),
    Datetime_First_End_Shift=('Datetime_End_Action', 'max')
).reset_index()
min_max_first_shifts['First Shift'] = (min_max_first_shifts['Datetime_First_Start_Shift'].dt.strftime('%H%M') + '-' + min_max_first_shifts['Datetime_First_End_Shift'].dt.strftime('%H%M'))
new_table_with_first_shifts_col = pd.merge(new_table_with_fluctuate_shifts_col, min_max_first_shifts, on=['Date', 'IEX_ID'], how='left')

filtered_original_shifts = filtered_max_generate_date[
    filtered_max_generate_date['Scheduled Activity'].isin(["Open Time"]) | 
    filtered_max_generate_date['Scheduled Activity'].str.contains("Training", na=False)
]
min_max_original_shifts = filtered_original_shifts.groupby(['Date', 'IEX_ID']).agg(
    Datetime_Original_Start_Shift=('Datetime_Start_Action', 'min'),
    Datetime_Original_End_Shift=('Datetime_End_Action', 'max')
).reset_index()
min_max_original_shifts['Original Shift'] = (min_max_original_shifts['Datetime_Original_Start_Shift'].dt.strftime('%H%M') + '-' + min_max_original_shifts['Datetime_Original_End_Shift'].dt.strftime('%H%M'))
new_table_with_original_shifts_col = pd.merge(new_table_with_first_shifts_col, min_max_original_shifts, on=['Date', 'IEX_ID'], how='left')

new_table_with_original_shifts_col = new_table_with_original_shifts_col[
    ~(
        (new_table_with_original_shifts_col['Scheduled Activity'].isin(["Lunch", "Break"])) & 
        new_table_with_original_shifts_col['Fluctuate Shift'].isna()
    )
]

new_table_with_original_shifts_col

,Agent,Date,Start_Shift,End_Shift,Scheduled Activity,Start_Action,End_Action,sheet_name,Export time,Generate Date,...,Datetime_End_Action,Datetime_Fluctuate_Start_Shift,Datetime_Fluctuate_End_Shift,Fluctuate Shift,Datetime_First_Start_Shift,Datetime_First_End_Shift,First Shift,Datetime_Original_Start_Shift,Datetime_Original_End_Shift,Original Shift
0,"Agent: 3001610 BUI, LYNHATDUY",2024-10-07,NaT,NaT,Off,NaT,NaT,VNM_07_Oct_13_Oct_2024,2024-10-17 00:49:06,2024-10-16 17:44:00,...,NaT,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN
1,"Agent: 3001610 BUI, LYNHATDUY",2024-10-08,NaT,NaT,Off,NaT,NaT,VNM_07_Oct_13_Oct_2024,2024-10-17 00:49:06,2024-10-16 17:44:00,...,NaT,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN
2,"Agent: 3003466 BUI, MINHTRIET",2024-10-08,NaT,NaT,Off,NaT,NaT,VNM_07_Oct_13_Oct_2024,2024-10-17 00:49:06,2024-10-16 17:44:00,...,NaT,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN
3,"Agent: 3003466 BUI, MINHTRIET",2024-10-09,NaT,NaT,Off,NaT,NaT,VNM_07_Oct_13_Oct_2024,2024-10-17 00:49:06,2024-10-16 17:44:00,...,NaT,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN
4,"Agent: 3083538 CAO, CHAUTOAN",2024-10-09,NaT,NaT,Off,NaT,NaT,VNM_07_Oct_13_Oct_2024,2024-10-17 00:49:06,2024-10-16 17:44:00,...,NaT,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44263,"Agent: 3001034 tran, thihaiyen",2024-09-14,04:00:00,13:00:00,Open Time,06:30:00,09:30:00,VNM_9_Sep_15_Sep_2024,2024-10-17 00:51:22,2024-10-16 17:46:00,...,2024-09-14 09:30:00,2024-09-14 04:00:00,2024-09-14 13:00:00,0400-1300,2024-09-14 04:00:00,2024-09-14 13:00:00,0400-1300,2024-09-14 04:00:00,2024-09-14 13:00:00,0400-1300
44264,"Agent: 3001034 tran, thihaiyen",2024-09-14,04:00:00,13:00:00,Lunch,09:30:00,10:30:00,VNM_9_Sep_15_Sep_2024,2024-10-17 00:51:22,2024-10-16 17:46:00,...,2024-09-14 10:30:00,2024-09-14 04:00:00,2024-09-14 13:00:00,0400-1300,2024-09-14 04:00:00,2024-09-14 13:00:00,0400-1300,2024-09-14 04:00:00,2024-09-14 13:00:00,0400-1300
44265,"Agent: 3001034 tran, thihaiyen",2024-09-14,04:00:00,13:00:00,Open Time,10:30:00,11:45:00,VNM_9_Sep_15_Sep_2024,2024-10-17 00:51:22,2024-10-16 17:46:00,...,2024-09-14 11:45:00,2024-09-14 04:00:00,2024-09-14 13:00:00,0400-1300,2024-09-14 04:00:00,2024-09-14 13:00:00,0400-1300,2024-09-14 04:00:00,2024-09-14 13:00:00,0400-1300
44266,"Agent: 3001034 tran, thihaiyen",2024-09-14,04:00:00,13:00:00,Break,11:45:00,12:00:00,VNM_9_Sep_15_Sep_2024,2024-10-17 00:51:22,2024-10-16 17:46:00,...,2024-09-14 12:00:00,2024-09-14 04:00:00,2024-09-14 13:00:00,0400-1300,2024-09-14 04:00:00,2024-09-14 13:00:00,0400-1300,2024-09-14 04:00:00,2024-09-14 13:00:00,0400-1300


In [8]:
sorted_generate_date = new_table_with_original_shifts_col.sort_values(by='Export time', ascending=True)

sorted_generate_date['Night_Shift'] = sorted_generate_date['Datetime_First_Start_Shift'].apply(lambda x: 1 if pd.to_datetime(x).hour >= 18 else 0)
sorted_generate_date['Week_Monday'] = sorted_generate_date['Date'] - pd.to_timedelta(sorted_generate_date['Date'].dt.weekday, unit='d')
sorted_generate_date['Week Begin'] = sorted_generate_date['Week_Monday'].dt.strftime('WB%d%m')
sorted_generate_date['Agent Name'] = sorted_generate_date['Agent'].str.extract(r'\d+ (.+)', expand=False).str.upper()
sorted_generate_date['Duration'] = (sorted_generate_date['Datetime_End_Action'] - sorted_generate_date['Datetime_Start_Action']).dt.total_seconds()

def calculate_total_time(df, activity_list, time_column_name):
    time_totals = df[df['Scheduled Activity'].isin(activity_list)]
    time_totals = time_totals.groupby(['Date', 'IEX_ID'])['Duration'].sum().reset_index()
    time_totals.rename(columns={'Duration': time_column_name}, inplace=True)
    time_totals[time_column_name] = time_totals[time_column_name] / 3600
    return time_totals

open_time_totals = calculate_total_time(sorted_generate_date, ['Open Time'], 'Open Time')
sorted_generate_date = sorted_generate_date.merge(open_time_totals, on=['Date', 'IEX_ID'], how='left')

rest_time_totals = calculate_total_time(sorted_generate_date, ['Break', 'Lunch'], 'Rest Time')
sorted_generate_date = sorted_generate_date.merge(rest_time_totals, on=['Date', 'IEX_ID'], how='left')

extra_time_totals = calculate_total_time(sorted_generate_date, ['Extra Hours'], 'Extra Time')
sorted_generate_date = sorted_generate_date.merge(extra_time_totals, on=['Date', 'IEX_ID'], how='left')

ncns_time_totals = calculate_total_time(sorted_generate_date, ['No Call/No Show'], 'NCNS')
sorted_generate_date = sorted_generate_date.merge(ncns_time_totals, on=['Date', 'IEX_ID'], how='left')

training_time_totals = calculate_total_time(sorted_generate_date, ['Training Offline','Training','Training Nesting'], 'Training')
sorted_generate_date = sorted_generate_date.merge(training_time_totals, on=['Date', 'IEX_ID'], how='left')

sorted_generate_date = sorted_generate_date.sort_values(by=['Date', 'IEX_ID', 'Datetime_Start_Action'])
sorted_generate_date['First_Scheduled_Activity'] = sorted_generate_date.groupby(['Date', 'IEX_ID'])['Scheduled Activity'].transform('first')

sorted_generate_date['Shift Tracking'] = np.where(
    (sorted_generate_date['Open Time'] > 0) & (sorted_generate_date['NCNS'] > 3),
    "HDL",
    np.where(
        ((sorted_generate_date['Open Time'] > 0) | (sorted_generate_date['Open Time'].isna())) & (sorted_generate_date['NCNS'] > 3),
        "UPL",
        np.where(
            (sorted_generate_date['Open Time'] > 0) & (sorted_generate_date['Extra Time'] > 0),
            "PR - OT",
            np.where(
                (sorted_generate_date['Open Time'] == 0) & (sorted_generate_date['Extra Time'] > 0),
                "PO",
                np.where(
                    (sorted_generate_date['Open Time'] > 0),
                    "PR",
                    np.where(
                        sorted_generate_date['First_Scheduled_Activity'] == "Extra Hours",
                        np.where(
                            sorted_generate_date['Open Time'].isna(),
                            "PO",
                            np.nan
                        ),
                        np.where(
                            sorted_generate_date['First_Scheduled_Activity'].isin(["Holiday", "Bereavement", "Off", "Off Phone Misc", "Termination", "Unscheduled"]),
                            sorted_generate_date['First_Scheduled_Activity'],
                            np.where(
                                sorted_generate_date['First_Scheduled_Activity'] == "PTO",
                                "AL",
                                np.where(
                                    sorted_generate_date['First_Scheduled_Activity'] == "Sickness",
                                    "SL",
                                    np.nan
                                )
                            )
                        )
                    )
                )
            )
        )
    )
)

In [9]:
def calculate_ot_pre_post_shift(df):
    extra_hours_df = df[df['Scheduled Activity'] == 'Extra Hours']
    
    ot_df = df[['Date', 'IEX_ID']].drop_duplicates()
    
    def calculate_duration_for_group(group_df, date, iex_id, shift_start, shift_end):
        pre_shift_duration = extra_hours_df[
            (extra_hours_df['Date'] == date) & 
            (extra_hours_df['IEX_ID'] == iex_id) & 
            (extra_hours_df['Datetime_Start_Action'] < shift_start)
        ]['Duration'].sum()
        
        post_shift_duration = extra_hours_df[
            (extra_hours_df['Date'] == date) & 
            (extra_hours_df['IEX_ID'] == iex_id) & 
            (extra_hours_df['Datetime_End_Action'] > shift_end)
        ]['Duration'].sum()
        
        return pre_shift_duration, post_shift_duration
    
    ot_df[['OT PreShift Seconds', 'OT PostShift Seconds']] = ot_df.apply(
        lambda row: pd.Series(
            calculate_duration_for_group(
                extra_hours_df, 
                row['Date'], 
                row['IEX_ID'], 
                df[(df['Date'] == row['Date']) & (df['IEX_ID'] == row['IEX_ID'])]['Datetime_Original_Start_Shift'].min(),
                df[(df['Date'] == row['Date']) & (df['IEX_ID'] == row['IEX_ID'])]['Datetime_Original_End_Shift'].max()
            )
        ), 
        axis=1
    )
    ot_df['OT PreShift'] = ot_df['OT PreShift Seconds'] / 3600
    ot_df['OT PostShift'] = ot_df['OT PostShift Seconds'] / 3600
    
    ot_df = ot_df.drop(columns=['OT PreShift Seconds', 'OT PostShift Seconds'])
    df = df.merge(ot_df, on=['Date', 'IEX_ID'], how='left')
    
    return df

OT_Pre_Post_Table = calculate_ot_pre_post_shift(sorted_generate_date)
OT_Pre_Post_Table = OT_Pre_Post_Table.groupby(['Date', 'IEX_ID'])[['OT PreShift', 'OT PostShift']].max().reset_index()

combine_OT_Pre_Post_Table = pd.merge(sorted_generate_date, OT_Pre_Post_Table, on=['Date','IEX_ID'], how='left')
combine_OT_Pre_Post_Table['Start Date'] = pd.to_datetime(combine_OT_Pre_Post_Table['Datetime_Start_Action']).dt.normalize()
combine_OT_Pre_Post_Table['End Date'] = pd.to_datetime(combine_OT_Pre_Post_Table['Datetime_End_Action']).dt.normalize()

In [10]:
# Select the desired columns
iex_ids_with_max_export_time = combine_OT_Pre_Post_Table[['sheet_name', 'Generate Date','Year','Month','Week Begin','Date','Start Date','End Date','Agent Name', 'IEX_ID','Datetime_Fluctuate_Start_Shift',
       'Datetime_Fluctuate_End_Shift', 'Fluctuate Shift',
       'Datetime_First_Start_Shift', 'Datetime_First_End_Shift',
       'First Shift','Scheduled Activity','Datetime_Start_Action','Datetime_End_Action','Duration','Open Time','Extra Time','Rest Time','Training','NCNS','Night_Shift', 'Scheduled','Shift Tracking','OT PreShift', 'OT PostShift']]

sorted_df = iex_ids_with_max_export_time.sort_values(
    by=[ 'Date','IEX_ID','Datetime_Start_Action'],
    ascending=[ True, True,True]
).reset_index(drop=True)

sorted_df = sorted_df.drop_duplicates()

def label_breaks(group):
    group.loc[group['Scheduled Activity'] == 'Break', 'Scheduled Activity'] = (
        group['Scheduled Activity'] + '_' + (group['Scheduled Activity'].eq('Break').cumsum()).astype(str)
    )
    return group

sorted_df = sorted_df.groupby(['Date', 'IEX_ID']).apply(label_breaks)
sorted_df = sorted_df.reset_index(drop=True)

sorted_df['Date'] = sorted_df['Date'].dt.date

C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25864\1085749366.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_df = sorted_df.groupby(['Date', 'IEX_ID']).apply(label_breaks)


In [11]:
split_table = sorted_df[['Date', 'Start Date', 'End Date', 'IEX_ID', 'Agent Name', 'Scheduled Activity', 'Datetime_Start_Action', 'Datetime_End_Action']]

for col in ['Start', 'End']:
    split_table[f'{col}_Time_Hours'] = split_table[f'Datetime_{col}_Action'].dt.hour + \
                                       split_table[f'Datetime_{col}_Action'].dt.minute / 60 + \
                                        split_table[f'Datetime_{col}_Action'].dt.second / 3600
    split_table[f'{col}_Time_Hours'] = split_table[f'{col}_Time_Hours'].round(2)

split_table['End_Time_Hours'] = split_table.apply(
    lambda row: row['End_Time_Hours'] + 24 if row['End_Time_Hours'] < row['Start_Time_Hours'] else row['End_Time_Hours'],
    axis=1
)

for col in ['Start', 'End']:
    split_table[f'{col}_Time_Half_Rounding'] = split_table[f'{col}_Time_Hours'].apply(
        lambda x: np.round(x * 2) / 2 if np.round(x * 2) / 2 == x else np.floor(x * 2) / 2
    )

split_table['Number_Split'] = split_table.apply(
    lambda row: (row['End_Time_Half_Rounding'] - row['Start_Time_Half_Rounding']) * 2 
    if pd.notna(row['End_Time_Half_Rounding']) and pd.notna(row['Start_Time_Half_Rounding']) 
    else np.nan,
    axis=1
)

split_table = split_table.rename(columns={'Start_Time_Half_Rounding': 'Time_1'})
split_table['Time_2'] = split_table['Time_1'] + 0.5
split_table['Custom'] = split_table['Number_Split'].apply(lambda n: list(range(int(n) + 1)) if pd.notna(n) else [])
split_table = split_table.explode('Custom')

split_table['Time_1_New'] = split_table['Time_1'] + (split_table['Custom'] * 0.5)
split_table['Time_2_New'] = split_table['Time_2'] + (split_table['Custom'] * 0.5)
split_table['Rounded_End_Time_Hours'] = np.ceil(split_table['End_Time_Hours'] * 2) / 2

filtered_df = split_table[split_table['Rounded_End_Time_Hours'] > split_table['Time_1_New']]

split_table['Start_Time_Number'] = np.where(
    split_table['Time_1_New'] > split_table['Start_Time_Hours'],
    split_table['Time_1_New'],
    split_table['Start_Time_Hours']
)

split_table['End_Time_Number'] = np.where(
    split_table['Time_2_New'] < split_table['End_Time_Hours'],
    split_table['Time_2_New'],
    split_table['End_Time_Hours']
)

def calculate_new_day_time(row, time_col, date_col):
    calculated_value = row[time_col]
    if row['Start Date'] == row['End Date']:
        return row[date_col]
    else:
        if time_col == 'Start_Time_Number' and calculated_value >= 24:
            return row['Start Date'] + pd.Timedelta(days=1)
        elif time_col == 'End_Time_Number' and calculated_value < 24:
            return row['End Date'] - pd.Timedelta(days=1)
        else:
            return row[date_col]

split_table['New_Day_Start_Time'] = split_table.apply(lambda row: calculate_new_day_time(row, 'Start_Time_Number', 'Start Date'), axis=1)
split_table['New_Day_End_Time'] = split_table.apply(lambda row: calculate_new_day_time(row, 'End_Time_Number', 'End Date'), axis=1)

for col in ['Start', 'End']:
    split_table[f'{col}_Time_Number'] = (split_table[f'{col}_Time_Number'] - 24 * (split_table[f'{col}_Time_Number'] >= 24)) % 24

def number_to_time(number):
    if pd.isna(number):
        return pd.NaT
    hours = int(np.floor(number))
    minutes = int(np.floor((number - hours) * 60))
    seconds = int(np.floor((number - hours - minutes / 60) * 3600))
    return pd.Timestamp(f"1970-01-01 {hours:02}:{minutes:02}:{seconds:02}")

for col in ['Start', 'End']:
    split_table[f'{col}_Time'] = split_table[f'{col}_Time_Number'].apply(number_to_time).dt.time

def combine_date_time(row, time_col, date_col):
    if pd.isna(row[time_col]):
        return pd.NaT
    return datetime.combine(row[date_col].date(), row[time_col])

split_table['Datetime_Start_Time'] = split_table.apply(lambda row: combine_date_time(row, 'Start_Time', 'New_Day_Start_Time'), axis=1)
split_table['Datetime_End_Time'] = split_table.apply(lambda row: combine_date_time(row, 'End_Time', 'New_Day_End_Time'), axis=1)

split_table = split_table[split_table['Datetime_Start_Time'] < split_table['Datetime_End_Time']]
split_table = split_table[['Date', 'Start Date', 'End Date', 'IEX_ID', 'Agent Name', 'Scheduled Activity', 'Datetime_Start_Action', 'Datetime_End_Action', 'Datetime_Start_Time', 'Datetime_End_Time']]

C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25864\1231678368.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_table[f'{col}_Time_Hours'] = split_table[f'Datetime_{col}_Action'].dt.hour + \
C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25864\1231678368.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  split_table[f'{col}_Time_Hours'] = split_table[f'{col}_Time_Hours'].round(2)
C:\Users\huuchinh.nguyen\AppData\Local\Temp\ipykernel_25864\1231678368.py:4: SettingWithCopyWarning: 
A 

In [50]:
import xlsxwriter
from datetime import date

all_sheets = {'1. INTERVALS_EXTEND': split_table,
            '2. IEX_EXTEND': sorted_df}

today_temp = datetime.today().date()
today = today_temp.strftime('%b_%d_%Y')

writer = pd.ExcelWriter(f'3. OUTPUT/{today}_IEX_DAILY_OUTPUT.xlsx', engine='xlsxwriter')
for sheet_name in all_sheets.keys():
    all_sheets[sheet_name].to_excel(writer, sheet_name=sheet_name, index=False)

writer.close()
today_temp

datetime.date(2024, 10, 17)